In [1]:
import os
import time
import random
import tensorflow as tf

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
from tensorflow.keras.datasets import mnist

os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [2]:
#set up the training and testing data
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = [x/255.0 for x in [x_train, x_test]]
y_train, y_test = [tf.keras.utils.to_categorical(y) for y in [y_train, y_test]]

In [3]:
#for more on the Adam Optimizer: https://keras.io/api/optimizers/adam/
opt = Adam()

In [22]:
class Discriminator:
    def __init__(self):
        dm = in1 = Input((28,28))
        dm = Reshape((28,28,1))(dm)
        
        dm = Conv2D(64, (5,5), padding='same', strides=(2,2))(dm)
        dm = BatchNormalization()(dm)
        dm = ELU()(dm)
        
        dm = Conv2D(128, (5,5), padding='same', strides=(2,2))(dm)
        dm = BatchNormalization()(dm)
        dm = ELU()(dm)
        
        dm = Flatten()(dm)
        dm = Dense(128)(dm)
        dm = BatchNormalization()(dm)
        dm = ELU()(dm)
        dm = Dense(1, activation='sigmoid')(dm)
        # Compile the model
        self.model = Model(in1, dm)
        self.model.compile(opt, 'binary_crossentropy')
        self.model.summary()

In [24]:
class Generator:
    def __init__(self, ZDIM):
        gm = in1 = Input((ZDIM,))
        
        gm = Dense(7*7*64)(gm)
        gm = BatchNormalization()(gm)
        gm = ELU()(gm)
        gm = Reshape((7,7,64))(gm)
        
        gm = Conv2DTranspose(128, (5,5), strides=(2,2), padding='same')(gm)
        gm = BatchNormalization()(gm)
        gm = ELU()(gm)
        
        gm = Conv2DTranspose(1, (5,5), strides=(2,2), padding='same')(gm)
        gm = Activation('sigmoid')(gm)
        gm = Reshape((28,28))(gm)
        # Compile the model
        self.model = Model(in1, gm)
        self.model.compile('adam', 'mse')
        self.model.summary()

In [26]:
x = Discriminator()
y = Generator(100)

Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_11 (InputLayer)        [(None, 28, 28)]          0         
_________________________________________________________________
reshape_8 (Reshape)          (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 14, 14, 64)        1664      
_________________________________________________________________
batch_normalization_20 (Batc (None, 14, 14, 64)        256       
_________________________________________________________________
elu_20 (ELU)                 (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 7, 7, 128)         204928    
_________________________________________________________________
batch_normalization_21 (Batc (None, 7, 7, 128)         512 